In [1142]:
import numpy as np 
import os 
import torch
import open3d as o3d
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time
import numpy as np 
import torch.nn as nn
import torch.nn.functional as F 

In [1143]:
path = "/media/parvez/One_Touch/ScanNet/scans/scene0000_00/scene0000_00_vh_clean_2.ply"



In [1144]:
pcd = o3d.io.read_point_cloud(path)


In [1145]:
#o3d.visualization.draw_geometries([pcd],
#                                 )

In [1146]:
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

In [1147]:
print(points.shape)

(81369, 3)


In [1148]:
def timeit(tag, t):
    print("{}: {}s".format(tag, time() - t))
    return time()

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.

    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst

    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist



def index_points(points, idx):
    """

    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S]
    Return:
        new_points:, indexed points data, [B, S, C]
    """
    device = points.device
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1 
    idx = idx.to(torch.long)
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    new_points = points[batch_indices, idx, :]

    #print("batch shape=", batch_indices.shape, "idx shape=", idx.shape, "points = ", points.shape, "new points=", new_points.shape)
    return new_points

 

def farthest_point_sample(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, point]
    """

    
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.float64).to(device)
    distance = torch.ones(B, N, dtype=torch.float64).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest 
        #print(xyz[batch_indices, farthest, :].shape, "in fps")
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance 
        dist = dist.to(torch.float64)
        #print(dist, "dist")
        #print(distance, "distance")
        distance[mask] = dist[mask]
        farthest = torch.max(distance, -1)[1]
    return centroids


def query_ball_point(radius, nsample, xyz, new_xyz):
    """
    Input:
        radius: local region radius
        nsample: max sample number in local region
        xyz: all points, [B, N, 3]
        new_xyz: query points, [B, S, 3]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    device = xyz.device
    B, N, C = xyz.shape
    _, S, _ = new_xyz.shape
    group_idx = torch.arange(N, dtype=torch.long).to(device).view(1, 1, N).repeat([B, S, 1])
    sqrdists = square_distance(new_xyz, xyz)
    group_idx[sqrdists > radius ** 2] = N
    group_idx = group_idx.sort(dim=-1)[0][:, :, :nsample]
    group_first = group_idx[:, :, 0].view(B, S, 1).repeat([1, 1, nsample])
    mask = group_idx == N
    group_idx[mask] = group_first[mask]
    return group_idx

In [1149]:
def sample_and_group(npoint, radius, nsample, xyz, points, returnfps=False):
    """
    Input:
        npoint:
        radius:
        nsample:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, npoint, nsample, 3]
        new_points: sampled points data, [B, npoint, nsample, 3+D]
    """
    B, N, C = xyz.shape
    S = npoint
    fps_idx = farthest_point_sample(xyz, npoint) # [B, npoint, C]
    torch.cuda.empty_cache()
    new_xyz = index_points(xyz, fps_idx)
    torch.cuda.empty_cache()
    idx = query_ball_point(radius, nsample, xyz, new_xyz)   # idx can have outlier index like max size
    torch.cuda.empty_cache()
    grouped_xyz = index_points(xyz, idx) # [B, npoint, nsample, C]   # verify shape
    torch.cuda.empty_cache()
    grouped_xyz_norm = grouped_xyz - new_xyz.view(B, S, 1, C)
    torch.cuda.empty_cache()

    #print("in sample and group all")
    if points is not None:
        grouped_points = index_points(points, idx)
        new_points = torch.cat([grouped_xyz_norm, grouped_points], dim=-1) # [B, npoint, nsample, C+D]
    else:
        new_points = grouped_xyz_norm
    if returnfps:
        return new_xyz, new_points, grouped_xyz, fps_idx
    else:
        return new_xyz, new_points


In [1150]:
class PointNetSetAbstraction(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp, group_all):
        super(PointNetSetAbstraction, self).__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel
        self.group_all = group_all

    def forward(self, xyz, points):
        """
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        """
        #print(xyz.shape, "XYZ")
        xyz = xyz.permute(0, 2, 1)
        if points is not None:
            points = points.permute(0, 2, 1)

        if self.group_all:
            new_xyz, new_points = sample_and_group_all(xyz, points)
        else:
            new_xyz, new_points = sample_and_group(self.npoint, self.radius, self.nsample, xyz, points)
        # new_xyz: sampled points position data, [B, npoint, C]
        # new_points: sampled points data, [B, npoint, nsample, C+D]
        #print("new_ponts before",  print("hello")new_points.shape)
        #print("hello")
        new_points = new_points.permute(0, 3, 2, 1).float() # [B, C+D, nsample,npoint]

        #print("hello Point abstraction")
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points =  F.relu(bn(conv(new_points)))
        #print(new_points.shape, "new points")
        new_points = torch.max(new_points, 2)[0]
        new_xyz = new_xyz.permute(0, 2, 1)
        return new_xyz, new_points


In [ ]:
class get_model(nn.Module):
    def __init__(self, num_classes):
        super(get_model, self).__init__()
        self.sa1 = PointNetSetAbstraction(1024, 0.1, 32, 9+3, [32, 32, 64], False)
        self.sa2 = PointNetSetAbstraction(256, 0.2, 32, 64 + 3, [64, 64, 128], False)
        self.sa3 = PointNetSetAbstraction(64, 0.4, 32, 128 + 3, [128, 128, 256], False)
        self.sa4 = PointNetSetAbstraction(16, 0.8, 32, 256 + 3, [256, 256, 512], False)
        self.conv1 = nn.Conv1d(512, 128, 1)
        self.bn1 = nn.BatchNorm1d(128)
        self.drop1 = nn.Dropout(0.5)
        self.conv2 = nn.Conv1d(128, num_classes, 1)

    def forward(self, xyz):
        l0_points = xyz
        l0_xyz = xyz[:,:3,:]


        l1_xyz, l1_points = self.sa1(l0_xyz, l0_points) 
        print(l1_xyz.shape, l1_points.shape, "sa1")
        l2_xyz, l2_points = self.sa2(l1_xyz, l1_points)
        print(l2_xyz.shape, l2_points.shape, "sa2")
        l3_xyz, l3_points = self.sa3(l2_xyz, l2_points)
        print(l3_xyz.shape, l3_points.shape, "sa3")
        l4_xyz, l4_points = self.sa4(l3_xyz, l3_points)
        print(l4_xyz.shape, l4_points.shape, "sa4")



        x = self.drop1(F.relu(self.bn1(self.conv1(l4_points))))
        x = self.conv2(x)
        x = F.log_softmax(x, dim=1)
        x = x.permute(0, 2, 1)
        return x, l4_points



In [1139]:
model = get_model(13)


In [1140]:
model

get_model(
  (sa1): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(12, 32, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa2): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(67, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa3): PointNetSetAbstraction(
  

In [1141]:
xyz = torch.rand(6, 9, 2048)
x, l4_points = model(xyz)

torch.Size([6, 3, 1024]) torch.Size([6, 64, 1024]) sa1
torch.Size([6, 3, 256]) torch.Size([6, 128, 256]) sa2
torch.Size([6, 3, 64]) torch.Size([6, 256, 64]) sa3
torch.Size([6, 3, 16]) torch.Size([6, 512, 16]) sa4
torch.Size([6, 64, 16]) dists
torch.Size([6, 256, 64]) dists
torch.Size([6, 1024, 256]) dists
torch.Size([6, 2048, 1024]) dists


In [1100]:
x.shape

torch.Size([6, 2048, 13])

In [1071]:
l4_points.shape

torch.Size([6, 512, 16])

In [1068]:
points = torch.from_numpy(points).unsqueeze(dim=0).permute(0,2,1).to(torch.long)

tensor([[37., 37., 37.,  ...,  6.,  6.,  6.]], dtype=torch.float64) dist
tensor([[1.0000e+10, 1.0000e+10, 1.0000e+10,  ..., 1.0000e+10, 1.0000e+10,
         1.0000e+10]], dtype=torch.float64) distance
tensor([[ 8.,  8.,  8.,  ..., 53., 53., 53.]], dtype=torch.float64) dist
tensor([[37., 37., 37.,  ...,  6.,  6.,  6.]], dtype=torch.float64) distance
tensor([[21., 21., 21.,  ..., 52., 52., 52.]], dtype=torch.float64) dist
tensor([[8., 8., 8.,  ..., 6., 6., 6.]], dtype=torch.float64) distance
tensor([[45., 45., 45.,  ..., 20., 20., 20.]], dtype=torch.float64) dist
tensor([[8., 8., 8.,  ..., 6., 6., 6.]], dtype=torch.float64) distance
tensor([[ 5.,  5.,  5.,  ..., 30., 30., 30.]], dtype=torch.float64) dist
tensor([[8., 8., 8.,  ..., 6., 6., 6.]], dtype=torch.float64) distance
tensor([[45., 45., 45.,  ..., 20., 20., 20.]], dtype=torch.float64) dist
tensor([[5., 5., 5.,  ..., 6., 6., 6.]], dtype=torch.float64) distance
tensor([[13., 13., 13.,  ..., 24., 24., 24.]], dtype=torch.float64) dist


RuntimeError: Given groups=1, weight of size [32, 12, 1, 1], expected input[1, 3, 32, 1024] to have 12 channels, but got 3 channels instead

In [ ]:
new_xyz, new_points = model(points, None)

In [741]:
new_xyz.shape

torch.Size([1, 3, 200])

In [742]:
new_points.shape

torch.Size([1, 64, 200])